In [1]:
import sys
import dis
import inspect

Generator functions can generate a function that behaves like an iterator. They allow programmers to make an iteraor in a fast, easy and clean way without too much memory cost. To illustrate this, let us consider a simple Python example of building a list and return it.

In [2]:
# firstn with list
def firstn(n):
    num, nums = 0, []
    while num < n:
        nums.append(num)
        num += 1
    return nums

By given an integer n, the function firstn return a full list with length n in memory. If n is really big number and each integer keeps 10 megabyte in memory, then we need to cost a lot of memory to extend our RAM. 

In [3]:
# firstn with generator pattern
class firstn1(object):
    def __init__(self, n):
        self.n = n
        self.num = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.num < self.n:
            cur, self.num = self.num, self.num + 1
        else:
            raise StopIteration()
        return cur

To save memory space, we can implement the firstn1 as an object with generator pattern. Class firstn1 is iterable and it will perform as we expect. However, we need to write a bunch lines of code to implement it and the logic is expressed in a convoluted way.

In [4]:
# firstn generator
def firstn2(n):
    num = 0
    while num < n:
        yield num
        num += 1

In [5]:
l, o, g = firstn(10000), firstn1(10000), firstn2(10000)
print(sys.getsizeof(l))  
print(sys.getsizeof(o)) 
print(sys.getsizeof(g))

87624
56
120


In [6]:
print(type(l))
print(type(o))
print(type(g)) 

<class 'list'>
<class '__main__.firstn1'>
<class 'generator'>


In [7]:
itr1, itr2, itr3 = iter(l), iter(o), iter(g)

In [8]:
print(sys.getsizeof(itr1))  
print(sys.getsizeof(itr2)) 
print(sys.getsizeof(itr3))

56
56
120


In [9]:
frame = None
def foo(x):
    global frame
    s = x
    frame = inspect.currentframe()
    return s
foo(1)

1

In [10]:
print("Frame:" ,frame)
print("Locals:",frame.f_locals)

Frame: <frame at 0x10c040540, file '<ipython-input-9-6df73ad12200>', line 6, code foo>
Locals: {'x': 1, 's': 1}


In [11]:
def foo(x,y):
    print('yielding x')
    yield x
    print ('yielding y')
    yield y
g = foo(1, 2)

In [12]:
dis.show_code(g)

Name:              foo
Filename:          <ipython-input-11-1e68f5d8ec40>
Argument count:    2
Kw-only arguments: 0
Number of locals:  2
Stack size:        2
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 'yielding x'
   2: 'yielding y'
Names:
   0: print
Variable names:
   0: x
   1: y


In [13]:
print('Last Instruction Pointer is at:', g.gi_frame.f_lasti)

Last Instruction Pointer is at: -1


In [14]:
dis.disco(g.gi_code, g.gi_frame.f_lasti)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('yielding x')
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_FAST                0 (x)
             10 YIELD_VALUE
             12 POP_TOP

  4          14 LOAD_GLOBAL              0 (print)
             16 LOAD_CONST               2 ('yielding y')
             18 CALL_FUNCTION            1
             20 POP_TOP

  5          22 LOAD_FAST                1 (y)
             24 YIELD_VALUE
             26 POP_TOP
             28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


In [15]:
next(g)

yielding x


1

In [16]:
print('Last Instruction Pointer is at:', g.gi_frame.f_lasti)

Last Instruction Pointer is at: 10


In [17]:
dis.disco(g.gi_code, g.gi_frame.f_lasti)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('yielding x')
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_FAST                0 (x)
    -->      10 YIELD_VALUE
             12 POP_TOP

  4          14 LOAD_GLOBAL              0 (print)
             16 LOAD_CONST               2 ('yielding y')
             18 CALL_FUNCTION            1
             20 POP_TOP

  5          22 LOAD_FAST                1 (y)
             24 YIELD_VALUE
             26 POP_TOP
             28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


In [18]:
next(g)

yielding y


2

In [19]:
print('Last Instruction Pointer is at:', g.gi_frame.f_lasti)

Last Instruction Pointer is at: 24


In [20]:
dis.disco(g.gi_code, g.gi_frame.f_lasti)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('yielding x')
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_FAST                0 (x)
             10 YIELD_VALUE
             12 POP_TOP

  4          14 LOAD_GLOBAL              0 (print)
             16 LOAD_CONST               2 ('yielding y')
             18 CALL_FUNCTION            1
             20 POP_TOP

  5          22 LOAD_FAST                1 (y)
    -->      24 YIELD_VALUE
             26 POP_TOP
             28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


In [21]:
next(g)

StopIteration: 